<a href="https://colab.research.google.com/github/SarathSabu/Python-Notebooks/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikeras
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.4.2

In [ ]:
import sklearn
import scikeras
print(sklearn.__version__)
print(scikeras.__version__)

In [ ]:
from tensorflow.keras import backend as K
# import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.random import set_seed
from random import seed
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# import data as dataframe
file_path = '/content/drive/MyDrive/Infor648/Data/churn_exam.csv'
df = pd.read_csv(file_path)

# calling head() method
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

#Data preprocessing

In [ ]:
display(df.isna().sum()) ##check missing value

In [ ]:
df = df.dropna() ##drop missing value
display(df.isna().sum()) ##recheck missing value again

In [ ]:
# Numeric Variables
numeric_variables = [col for col in df.columns if df[col].dtype != "object" and col not in "Customer Status"] ##exclude our target variable: customer status
numeric_variables

In [ ]:
categorical_variables = [col for col in df.columns if df[col].dtype == "O" and col != "Customer Status"]  ###exclude our target: "Customer Status"
categorical_variables

In [ ]:
df['Internet Type'].value_counts()

In [ ]:
df['Contract'].value_counts()

In [ ]:
df['Offer'].value_counts()

#Select Variables of Interest

In [ ]:
df_sub = df[["Number of Dependents", "Number of Referrals","Total Long Distance Charges","Total Extra Data Charges","Gender","Offer","Unlimited Data", "Customer Status"]]

In [ ]:
df_sub

In [ ]:
##encode categorical data
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_sub['Gender'] = label_encoder.fit_transform(df_sub['Gender'])
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))


#Print out what we encoded for gender
print("Gender Encoding:")
print(mapping)

In [ ]:
##encode categorical data
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_sub['Unlimited Data'] = label_encoder.fit_transform(df_sub['Unlimited Data'])
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))


#Print out what we encoded for gender
print("Unlimited Data Encoding:")
print(mapping)

#One-hot encoding

In [ ]:
# One-hot encoding transforms categorical variables into a set of binary columns (0s and 1s), one for each category.
# This ensures no ordinal relationship is imposed between the categories (which could be problematic in label encoding).

df_sub_encoded = pd.get_dummies(df_sub, columns=['Offer'])

# Ensure that all binary columns are integers (0 and 1)
df_sub_encoded = df_sub_encoded.astype({col: int for col in df_sub_encoded.columns if 'Offer' in col})



df_sub_encoded

#Encod our target variable

In [ ]:
display(df_sub_encoded['Customer Status'].value_counts())
##Our target variable is a categorical variable

In [ ]:
####Encode our target variable
target_label_encoder = LabelEncoder()
df_sub_encoded['Customer Status'] = target_label_encoder.fit_transform(df_sub_encoded['Customer Status'])


##display the stats after encoding
display(df_sub_encoded['Customer Status'].value_counts())
mapping = dict(zip(target_label_encoder.classes_, target_label_encoder.transform(target_label_encoder.classes_)))
print(mapping)

In [ ]:
corr_matrix = df_sub_encoded.corr()
plt.figure(figsize=(9,9)) ###change the figure size here
sns.heatmap(corr_matrix, cmap='Blues', annot=True)
plt.show()

#Split into test and training dataset

In [ ]:
X = df_sub_encoded.drop('Customer Status', axis=1)  # Drop the target column to get independent variables
y = df_sub_encoded['Customer Status']  # Select the target column directly as our y


# Split the dataset into training and testing sets test_size using 0.3: 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)



In [ ]:
X_train

In [ ]:
y_train

#Normalization or standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
#data standardization
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#data normalization
#nc = MinMaxScaler()
#X_train = nc.fit_transform(X_train)
#X_test = nc.transform(X_test)

In [ ]:
##you do not need to do anything here
def nn_train(num_layers = 2, num_neurons = 8, opt='adam', dropout_rate=0.0, seed_value=42, input_shape=None):
    """
    Build and train a neural network for binary classification.

    Parameters:
    input_shape: Input features for training.
    num_layers (int): Total number of hidden layers (excluding input/output layers).
    num_neurons (int): Number of neurons in each hidden layer.
    opt (str): Optimizer to use (default: 'adam').
    seed_value (int): Random seed value for reproducibility.
    dropout_rate (float): Fraction of input units to drop, between 0 and 1 (default: 0).
    randomly "drops" a fraction of the neurons' outputs in the layer it's applied to during each training step. This helps prevent overfitting
    """
    K.clear_session()
    # Set random seed for reproducibility
    seed(seed_value)
    set_seed(seed_value)

    # Initialize the Sequential model
    model = Sequential()

    # Add an Input layer to specify the input shape
    model.add(Input(shape=(input_shape,)))

    # Add additional hidden layers, all with `num_neurons` neurons
    for _ in range(num_layers):
      model.add(Dense(num_neurons, activation='relu'))
      if dropout_rate > 0:  # Add dropout only if rate is greater than 0
            model.add(Dropout(dropout_rate))

    # Output layer with 1 neuron for binary classification using sigmoid activation
    model.add(Dense(1, activation='sigmoid')) ###change this to softmax for multi-class and change the number here to align with the number of class
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy']) #change to categorical_crossentropy


    return model



In [ ]:
#num_layers (int): Total number of hidden layers (excluding input/output layers).
#num_neurons (int): Number of neurons in each hidden layer.
nn_model = nn_train(num_layers=3, input_shape=X_train.shape[1], num_neurons=8, opt='adam', seed_value=42) #You can heuristically set the number of neurons in each hidden layer to be 2 times the input dimension

In [ ]:
print(nn_model.summary())

##parameter calculation = #weights(#Input * #neurons) + #bias
#4 bytes per parameter

In [ ]:
#X_train: Input features for training.
#y_train: Target variable for training.

training = nn_model.fit(X_train, y_train, epochs=50, batch_size=30, validation_split=0.2) #split into validation set to evaluate how well the model performs on unseen data during training.
#Batch Size: 30 means that the model will update its parameters after every 30 sample(every batch)
#Epoch: meaning the model will go through the entire dataset 50 times.

In [ ]:
validation_accurancy = np.mean(training.history['val_accuracy'])
print("\n%s: %.2f%%" % ('validation_accurancy', validation_accurancy*100))

In [ ]:
##You do not need to do anything here
def plot_training_history(training):
    # Plot accuracy
    plt.figure(figsize=(12, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(training.history['accuracy'], label='Train Accuracy')
    plt.plot(training.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(training.history['loss'], label='Train Loss')
    plt.plot(training.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Show the plot
    plt.tight_layout()
    plt.show()



In [ ]:
plot_training_history(training)

#Fine-tunig

In [ ]:
# Wrap the function with KerasClassifier for grid search


from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(model=nn_train, input_shape=X_train.shape[1], verbose=0)
#model = KerasClassifier(model=nn_train, n_features=X_train.shape[1], verbose=0)
batch_size = [20, 30, 60] ##change the batch size here
epochs = [30, 50, 60]  #change the number of epochs here
#if you select more than 3 numbers, it will take some time to train
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Perform grid search using GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose = 2) #you can choose cv as 3 or 5
grid_result_batch = grid_search.fit(X_train, y_train)



In [ ]:
# summarize results
print(f"Best parameters: {grid_result_batch.best_params_}")
print(f"Best accuracy: {grid_result_batch.best_score_}")
means = grid_result_batch.cv_results_['mean_test_score']
stds = grid_result_batch.cv_results_['std_test_score']
params = grid_result_batch.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
model = KerasClassifier(model=nn_train, input_shape=X_train.shape[1], num_layers=None, num_neurons=None,
                        epochs=50, batch_size=60, verbose=0)

# define the grid search parameters
num_layers = [1, 2, 3]  # Different number of hidden layers (less than 5)
num_neurons =  [8, 16, 22]  # Different number of neurons, you can try 2*input dimension

param_grid = dict(num_layers=num_layers, num_neurons = num_neurons)

# search the grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv = 3, verbose=2) ##cv cross-validation 3-fold or 5-fold
grid_result_layer = grid.fit(X_train, y_train)

In [ ]:
# summarize results
print(f"Best parameters: {grid_result_layer.best_params_}")
print(f"Best accuracy: {grid_result_layer.best_score_}")
means = grid_result_layer.cv_results_['mean_test_score']
stds = grid_result_layer.cv_results_['std_test_score']
params = grid_result_layer.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:

model = KerasClassifier(model=nn_train, input_shape=X_train.shape[1], num_layers=2, num_neurons=16, dropout_rate= None,
                        epochs=50, batch_size=60, verbose=0)
# define the grid search parameters
dropout_rates = [0.0, 0.2, 0.3, 0.5]

param_grid_dropout = dict(dropout_rate = dropout_rates)

grid_dropout = GridSearchCV(estimator=model, param_grid=param_grid_dropout, cv=3, verbose=2)
grid_result_dropout = grid_dropout.fit(X_train, y_train)

# Get the best dropout rate
best_dropout_rate = grid_result_dropout.best_params_['dropout_rate']

In [ ]:
# summarize results
print(f"Best parameters: {grid_result_dropout.best_params_}")
print(f"Best accuracy: {grid_result_dropout.best_score_}")
means = grid_result_dropout.cv_results_['mean_test_score']
stds = grid_result_dropout.cv_results_['std_test_score']
params = grid_result_dropout.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# create final model
final_model = nn_train(num_layers=2, input_shape=X_train.shape[1], num_neurons=16, opt='adam', dropout_rate = 0.0, seed_value=42)

final_model.summary()

In [ ]:
final_training = final_model.fit(X_train, y_train, epochs=50, batch_size=60,
                     validation_split=0.2, verbose=2)

In [ ]:
# evaluate the model
validation_accurancy_final = np.mean(final_training.history['val_accuracy'])
print("\n%s: %.2f%%" % ('validation_accurancy_final', validation_accurancy_final*100))
print("\n%s: %.2f%%" % ('validation_accurancy_previous', validation_accurancy*100))

In [ ]:
plot_training_history(final_training)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the model on the test data
test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Make predictions on the test data
y_pred = final_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred[y_pred >0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred

In [ ]:

# Create a DataFrame for evaluation metrics
evaluation_metrics = pd.DataFrame({
    "Evaluation Metric": ["Train Accuracy", "Test Accuracy", "Recall", "Precision", "F1 Score"],
    "Value": [
        validation_accurancy_final * 100,
        accuracy_score(y_test, y_pred) * 100,
        recall_score(y_test, y_pred) * 100,
        precision_score(y_test, y_pred) * 100,
        f1_score(y_test, y_pred) * 100
    ]
})

# Display the DataFrame
display(evaluation_metrics)


In [ ]:
# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))  # Adjust figure size here
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.ylabel('True Class')
plt.xlabel('Predicted Class')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
X

Suppose we have a consumer with 2 dependents, 0 referral and 50.31 total long distance charges, 0 total extra data charges, male and yes for unlimited data and offered offer A

In [ ]:
# Create the input data for a single prediction (make sure it matches the encoding of your training data)
single_input = np.array([[2, 0, 50.31, 0, 1, 1, 1, 0, 0, 0, 0]])  # Replace these values with your actual input


single_input_scaled = sc.transform(single_input)
# Make a single prediction
single_prediction = final_model.predict(single_input_scaled)


In [ ]:
single_prediction

In [ ]:
single_prediction = (single_prediction > 0.5).astype("int32")

print(f"Predicted class for the input: {single_prediction[0][0]}")